In [1]:
import numpy as np 
import pandas as pd
import functools
import warnings
import holidays

from datetime import datetime


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
df_voos = pd.read_parquet('info_voos.parquet')
df_meteorologia = pd.read_parquet('info_meteorologia.parquet')

In [3]:
df_voos.head()

,IATA_origem,ICAO_origem,IATA_destino,ICAO_destino,data,partida_prevista,partida_real,chegada_prevista,chegada_real,cia_aerea,rota,duracao_prevista_voo,duracao_real_voo,atraso_chegada,atraso_partida,ano_partida,mes_partida,dia_partida,hora_partida,tempo_atraso,dia_semana,fim_de_semana,turno_partida,turno_chegada,feriado
0,REC,SBRF,POA,SBPA,01/01/2022,2022-01-01 17:45:00,2022-01-01 17:41:00,2022-01-01 22:05:00,2022-01-01 21:49:00,AZUL,REC_POA,-260.0,-248.0,False,False,2022,1,1,17,0 days 00:04:00,5,1,Tarde,Noite,1
1,REC,SBRF,POA,SBPA,02/01/2022,2022-01-02 17:25:00,2022-01-02 17:23:00,2022-01-02 21:50:00,2022-01-02 21:36:00,AZUL,REC_POA,-265.0,-253.0,False,False,2022,1,2,17,0 days 00:02:00,6,1,Tarde,Noite,0
2,REC,SBRF,POA,SBPA,03/01/2022,2022-01-03 02:25:00,2022-01-03 02:22:00,2022-01-03 06:45:00,2022-01-03 06:30:00,AZUL,REC_POA,-260.0,-248.0,False,False,2022,1,3,2,0 days 00:03:00,0,0,Madrugada,Manha,0
3,REC,SBRF,POA,SBPA,03/01/2022,2022-01-03 17:25:00,2022-01-03 17:22:00,2022-01-03 21:50:00,2022-01-03 21:29:00,AZUL,REC_POA,-265.0,-247.0,False,False,2022,1,3,17,0 days 00:03:00,0,0,Tarde,Noite,0
4,REC,SBRF,POA,SBPA,04/01/2022,2022-01-04 02:25:00,2022-01-04 02:20:00,2022-01-04 06:45:00,2022-01-04 06:29:00,AZUL,REC_POA,-260.0,-249.0,False,False,2022,1,4,2,0 days 00:05:00,1,0,Madrugada,Manha,0


In [4]:
df_meteorologia.head()

,time,temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,date,hour,elevation,weathercode,IATA,hora,data
0,2022-01-01T00:00,18.8,1.1,1.1,100,51,85,100,9.3,18.6,2022-01-01,00:00,1057,63,BSB,0,01/01/2022
1,2022-01-01T01:00,18.4,0.9,0.9,100,95,88,99,9.7,18.3,2022-01-01,01:00,1057,63,BSB,1,01/01/2022
2,2022-01-01T02:00,18.1,0.5,0.5,100,95,96,98,7.5,15.7,2022-01-01,02:00,1057,63,BSB,2,01/01/2022
3,2022-01-01T03:00,18.1,0.2,0.2,100,86,71,99,5.9,13.6,2022-01-01,03:00,1057,63,BSB,3,01/01/2022
4,2022-01-01T04:00,18.2,0.2,0.2,100,98,72,98,5.8,12.0,2022-01-01,04:00,1057,63,BSB,4,01/01/2022


In [5]:
df_full = df_voos.merge(df_meteorologia,
             left_on=['IATA_origem', 'data', 'hora_partida'],
             right_on=['IATA', 'data', 'hora'],
             how='left'
)

In [6]:
df_final = df_full[
    (~df_full["IATA"].isna()) &
    (df_full["cia_aerea"].isin(['AZUL', 'GOL', 'TAM']))
]

In [7]:
df_final.to_parquet('dataset_voos.parquet')